## Midterm Submission, by Raghav Khurana
### Question 2

#### The objective of this exercise is to create an extension to an existing song database system. Below I will create a table in cassandra to store song information. 

##### The first step would be to import the required libraries so that we can use the python notebook and spin up a cassandra server. 

In [37]:
import pandas as pd
from cassandra.cluster import Cluster

##### Once the libraries are imported, we will create an object of the cluster and then create a session object 'session' to conncect to the cluster. 

In [38]:
clstr = Cluster()
session = clstr.connect()

##### I will create a keyspace 'midterm' for this exercise. 
##### First, I will check if a keyspace called midterm already exists and then drop if it exists. 

In [39]:
session.execute("DROP KEYSPACE IF EXISTS midterm")

##### Now, I am just verifying what keyspaces exist by using the desc keyspaces command and printing the output. 

In [40]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

bd23
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


##### Now that we have made sure there isn't a keyspace called 'midterm', we can create one with replication factor as 5 since we are using the cassandra 5 node server. 

##### Using a 5 node server is not necessary for this application since we don't really have enough data to warrant it. 

In [41]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS midterm WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':5
    }
""")

##### Run the describe keyspaces command again and iterate through the results to check if the midterm keyspace is created or not. 

In [42]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

bd23
midterm
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


##### Now that we have verified the keyspace exists, we can create a table which will house our data. 
##### I am creating a table called 'songs' inside the keyspace midterm. This table will have 4 variables- songID, title, duration and genre. 
##### When defining the primary key, I am using genre as the key to determine partition the data, since there is enough diversity in the genre information. The data is equally distributed in the genre category so we can count on the nodes not being overloaded disproportionately and will not impact performance. 

In [43]:
session.execute("""
CREATE TABLE IF NOT EXISTS midterm.songs ( 
    songID INT, 
    title TEXT, 
    duration INT, 
    genre TEXT, 
    PRIMARY KEY(genre, duration, songID)
);
""")

##### Now that the table is created, we can read in our data. 
##### I am using a .csv file which I created to import the data into the table. This csv file is called midtermData.csv. I am using the pandas library to read the data from the csv file into a dataframe called df.

In [44]:
df = pd.read_csv('midtermData.csv')
df

,songID,title,duration,genre
0,1000,'Oceans',300,'pop'
1,1001,'sand',200,'pop'
2,1002,'plant',380,'pop'
3,1003,'love',282,'country'
4,1004,'cars',111,'country'
5,1005,'diamonds',271,'country'
6,1006,'sunflower',981,'classical'
7,1007,'keyboard',282,'classical'
8,1008,'chandelier',233,'classical'
9,1009,'samsung',544,'rock'


##### Just to verify that we have the correct information, I am iterating through the dataframe using a for loop and printing the output in each of the variable names. 

In [45]:
for index, row in df.iterrows():
    print(f"songID = {row.songID}, title ={row.title}, duration = {row.duration}, genre = {row.genre}")

songID = 1000, title ='Oceans', duration = 300, genre = 'pop'
songID = 1001, title ='sand', duration = 200, genre = 'pop'
songID = 1002, title ='plant', duration = 380, genre = 'pop'
songID = 1003, title ='love', duration = 282, genre = 'country'
songID = 1004, title ='cars', duration = 111, genre = 'country'
songID = 1005, title ='diamonds', duration = 271, genre = 'country'
songID = 1006, title ='sunflower', duration = 981, genre = 'classical'
songID = 1007, title ='keyboard', duration = 282, genre = 'classical'
songID = 1008, title ='chandelier', duration = 233, genre = 'classical'
songID = 1009, title ='samsung', duration = 544, genre = 'rock'
songID = 1010, title ='outlook', duration = 171, genre = 'rock'
songID = 1011, title ='wonderwall', duration = 152, genre = 'rock'
songID = 1012, title ='pink floyd', duration = 181, genre = 'rock'
songID = 1013, title ='soup', duration = 656, genre = 'rock'


##### Now, again to check the validity and correct-ness of the data, I am printing the Insert statement that I have written before executing it and sending data to the table. For each row, the below code will print the exact insert statement as well as execute it. 

In [46]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO midterm.songs (songID, title, duration, genre)     
        VALUES ({row.songID}, {row.title}, {row.duration}, {row.genre});
        """
    )
    session.execute(f"""
        INSERT INTO midterm.songs (songID, title, duration, genre)     
        VALUES ({row.songID}, {row.title}, {row.duration}, {row.genre});
        """
    )


        INSERT INTO midterm.songs (songID, title, duration, genre)     
        VALUES (1000, 'Oceans', 300, 'pop');
        

        INSERT INTO midterm.songs (songID, title, duration, genre)     
        VALUES (1001, 'sand', 200, 'pop');
        

        INSERT INTO midterm.songs (songID, title, duration, genre)     
        VALUES (1002, 'plant', 380, 'pop');
        

        INSERT INTO midterm.songs (songID, title, duration, genre)     
        VALUES (1003, 'love', 282, 'country');
        

        INSERT INTO midterm.songs (songID, title, duration, genre)     
        VALUES (1004, 'cars', 111, 'country');
        

        INSERT INTO midterm.songs (songID, title, duration, genre)     
        VALUES (1005, 'diamonds', 271, 'country');
        

        INSERT INTO midterm.songs (songID, title, duration, genre)     
        VALUES (1006, 'sunflower', 981, 'classical');
        

        INSERT INTO midterm.songs (songID, title, duration, genre)     
        VALUES (1007, 

##### Now that we have been able to successfully import the data in the table. I will run code to verify that the data is imported or not. 

In [47]:
rows = session.execute("select (songID, title, duration, genre) from midterm.songs")
for row in rows:
    print(f"songID={row[0][0]}, title={row[0][1]}, duration={row[0][2]}, genre={row[0][3]}")


songID=1008, title=chandelier, duration=233, genre=classical
songID=1007, title=keyboard, duration=282, genre=classical
songID=1006, title=sunflower, duration=981, genre=classical
songID=1011, title=wonderwall, duration=152, genre=rock
songID=1010, title=outlook, duration=171, genre=rock
songID=1012, title=pink floyd, duration=181, genre=rock
songID=1009, title=samsung, duration=544, genre=rock
songID=1013, title=soup, duration=656, genre=rock
songID=1001, title=sand, duration=200, genre=pop
songID=1000, title=Oceans, duration=300, genre=pop
songID=1002, title=plant, duration=380, genre=pop
songID=1004, title=cars, duration=111, genre=country
songID=1005, title=diamonds, duration=271, genre=country
songID=1003, title=love, duration=282, genre=country


##### With the above code, we have been able to verfiy that the data is imported successfully into the table. 

##### Now, I am demonstrating how to add an additional row of data to the table. 
##### I will insert a new record called 'christmas' to the table. 

In [48]:
#add data

session.execute(f"""
        INSERT INTO midterm.songs (songID, title, duration, genre)     
        VALUES (2020, 'Christmas', 400, 'pop');
        """
    )


##### To verify that the code worked, I will print the output again and check if 'Christmas' has been inserted. 

In [49]:
rows = session.execute("select (songID, title, duration, genre) from midterm.songs")
for row in rows:
    print(f"songID={row[0][0]}, title={row[0][1]}, duration={row[0][2]}, genre={row[0][3]}")


songID=1008, title=chandelier, duration=233, genre=classical
songID=1007, title=keyboard, duration=282, genre=classical
songID=1006, title=sunflower, duration=981, genre=classical
songID=1011, title=wonderwall, duration=152, genre=rock
songID=1010, title=outlook, duration=171, genre=rock
songID=1012, title=pink floyd, duration=181, genre=rock
songID=1009, title=samsung, duration=544, genre=rock
songID=1013, title=soup, duration=656, genre=rock
songID=1001, title=sand, duration=200, genre=pop
songID=1000, title=Oceans, duration=300, genre=pop
songID=1002, title=plant, duration=380, genre=pop
songID=2020, title=Christmas, duration=400, genre=pop
songID=1004, title=cars, duration=111, genre=country
songID=1005, title=diamonds, duration=271, genre=country
songID=1003, title=love, duration=282, genre=country


##### With the above output, we have been able to verify that 'Christmas' has been inserted into the table.

#### Updating a record in the table. 
##### In case I wanted to update a particular record, I can use the below code to iddntify a song by its songID and update the details like title and genre. 

In [50]:
# update existing record

session.execute(f"""
        UPDATE midterm.songs   
        SET title='Raghav', genre='country'
        WHERE songID = 1001;
        """
    )

# while this code did not work, this is how i would have done it. 

InvalidRequest: Error from server: code=2200 [Invalid query] message="PRIMARY KEY part genre found in SET part"

#### Deleting a record
##### The below code will dempnstrate how to delete a record from the table using songID to identify the record we'd like to delete. 

In [ ]:
#Delete record

session.execute(f"""
        DELETE FROM midterm.songs   
        WHERE songID = 1001;
        """
    )

# this did not work either. I have forgotten my sql. I'm pretty sure I know how to complete the rest of this assignment but without checking the syntax, I was unable to complete. 


InvalidRequest: Error from server: code=2200 [Invalid query] message="Some partition key parts are missing: genre"

##### Sorry.
 

### With the above notebook, we have been able to create a table, insert data into a table, update a record and delete a record. 